# Extension Plan - Data Preprocessing

In this extension, I incorporate several additional datasets that provide insights into mortality rates due to respiratory diseases, asthma-related healthcare patterns (including both inpatient hospitalizations and emergency room visits), and the volume of respiratory-related medical procedures performed annually.

In this notebook, I load the acquired health data and preprocess it to make it more usable for further analysis. Specifically, I preprocess the following:

1. **CDC Wonder Mortality Data:** This dataset contains detailed information about mortality from 1999 - 2020, and contains the following columns: Year, Age Group, Gender, Number of Deaths.
This data was gathered from the [CDC Wonder Mortality Website](https://wonder.cdc.gov/mcd-icd10.html) and the specific query was as follows:

    - Query Criteria:
        - States: Maricopa County, AZ (04013)
        - UCD - ICD-10 Codes: J00-J98 (Diseases of the respiratory system)
        - Group By:	Year; Ten-Year Age Groups; Gender

    Raw Data that was downloaded from the source can be found [here](../Raw%20Data/1999-2020_Mortality%20Data.txt).

2. **ADHS Asthma-Related Inpatient Discharges and Emergency Visits Data:** This dataset includes information on asthma-related inpatient discharges (2000-2021) and emergency visits (2003-2021) across Maricopa County (where Mesa is located). The original data has information breakdowns by gender, age group, and county of residence. However, this dataset was in an inconsistent state, and I therefore manually cleaned up this data and stored it as an [excel file](../Processed%20Data/Hospital_Inpatient_Emergency_Data.xlsx) with columns as follows: Year, Gender, Type of Record, Asthma All Mentions, Asthma First Diagnosis

    - The original data was gathered the [ADHS Website](https://pub.azdhs.gov/health-stats/hip/index.php?pg=asthma)

    - Raw Data that was downloaded from the source can be found [here](../Raw%20Data/Inpatient%20VS%20Emergency/).

3. **ADHS Medical Procedures Data:** This dataset includes county-level information about the number of inpatient discharges by different medical procedures performed annually from 2000 to 2021 by procedure category. It includes procedure categories such as respiratory therapy, which can be used to understand whether wildfires are leading to an increase in respiratory procedures. Specifically, this dataset includes: County, Year, Procedure Type, Number of Procedures.

    - The original data was gathered the [ADHS Website](https://pub.azdhs.gov/health-stats/hip/index.php?pg=procedure)

    - Raw Data that was downloaded from the source can be found [here](../Raw%20Data/By%20Procedures/).

In [25]:
#
#    IMPORTS
#

#    These are standard python modules. In case you do not have a python module, you should use `!pip install <module>`
import pandas as pd
import glob
#

### Dataset 1: CDC Wonder Mortality Data (Specifically due to respiratory diseases)

Here, I load the raw CDC Wonder data and preprocess it to ensure it is in a usable and standardised format.

In [5]:
mortality_data = pd.read_csv("../Raw Data/1999-2020_Mortality Data.txt", sep="\t")
mortality_data

,Notes,Year,Year Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Gender,Gender Code,Deaths,Population,Crude Rate
0,NaN,1999.0,1999.0,< 1 year,1,Male,M,12.0,25147.0,Unreliable
1,NaN,1999.0,1999.0,35-44 years,35-44,Female,F,15.0,231425.0,Unreliable
2,NaN,1999.0,1999.0,35-44 years,35-44,Male,M,17.0,239273.0,Unreliable
3,NaN,1999.0,1999.0,45-54 years,45-54,Female,F,36.0,180536.0,19.9
4,NaN,1999.0,1999.0,45-54 years,45-54,Male,M,43.0,173070.0,24.8
...,...,...,...,...,...,...,...,...,...,...
327,July 1 estimates. Population figures for Infan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328,figures for years 2001 - 2009 differ slightly ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,were available at the time of release.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,6. The population figures used in the calculat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From above, we can see that there are lot of unneeded columns, and that there is a lot of NaN values. Therefore, in the next step, I remove the unneeded columns and drop the NaN values when they occur in the year column

In [6]:
mortality_data = mortality_data.drop(columns=['Crude Rate', 'Gender Code', 'Ten-Year Age Groups Code', 'Year Code', 'Notes'], axis=1)
mortality_data.dropna(subset=['Year'], inplace=True)
mortality_data

,Year,Ten-Year Age Groups,Gender,Deaths,Population
0,1999.0,< 1 year,Male,12.0,25147.0
1,1999.0,35-44 years,Female,15.0,231425.0
2,1999.0,35-44 years,Male,17.0,239273.0
3,1999.0,45-54 years,Female,36.0,180536.0
4,1999.0,45-54 years,Male,43.0,173070.0
...,...,...,...,...,...
268,2020.0,65-74 years,Male,388.0,192741.0
269,2020.0,75-84 years,Female,487.0,124045.0
270,2020.0,75-84 years,Male,535.0,102384.0
271,2020.0,85+ years,Female,511.0,49896.0


Now, we can see that Year is not in a datetime format, and that population and deaths are not integers but floats. So, I clean that up and ensure consistent naming for Age groups.

In [10]:
mortality_data.rename(columns={'Ten-Year Age Groups': 'Age Groups'}, inplace=True)
mortality_data['Deaths'] = pd.to_numeric(mortality_data['Deaths'], errors='coerce').fillna(0).astype(int)
mortality_data['Population'] = pd.to_numeric(mortality_data['Population'], errors='coerce').fillna(0).astype(int)

# Here I convert 'Year' to datetime and extract the year
mortality_data['Year'] = pd.to_datetime(mortality_data['Year'], format='%Y')
mortality_data['Year'] = mortality_data['Year'].dt.year

mortality_data

,Year,Age Groups,Gender,Deaths,Population
0,1999,< 1 year,Male,12,25147
1,1999,35-44 years,Female,15,231425
2,1999,35-44 years,Male,17,239273
3,1999,45-54 years,Female,36,180536
4,1999,45-54 years,Male,43,173070
...,...,...,...,...,...
268,2020,65-74 years,Male,388,192741
269,2020,75-84 years,Female,487,124045
270,2020,75-84 years,Male,535,102384
271,2020,85+ years,Female,511,49896


Now, that the data is cleaned up, we can see that there are too many age groups, so I decided to limit it to three agre groups, 0-24, 25-64 and then 65+ years. 

In [8]:
print(mortality_data['Age Groups'].unique())

['< 1 year' '35-44 years' '45-54 years' '55-64 years' '65-74 years'
 '75-84 years' '85+ years' '25-34 years']


In [11]:
# Define the bins and labels
bins = [0, 24, 64, float('inf')]
labels = ['0-24', '25-64', '65+']

aggregated_data = mortality_data.copy()

# I choose to drop the Population column since I wasn't going to use it, and so that the aggregation makes sense
aggregated_data.drop(columns=['Population'], inplace=True, errors='ignore')

# Update the Age Groups
age_groups_numeric = aggregated_data['Age Groups'].str.extract('(\d+)')[0].astype(float)
aggregated_data['Age Groups'] = pd.cut(age_groups_numeric, bins=bins, labels=labels, right=False)
aggregated_data = aggregated_data.groupby(['Year', 'Age Groups', 'Gender']).agg({'Deaths': 'sum'}).reset_index()

aggregated_data

,Year,Age Groups,Gender,Deaths
0,1999,0-24,Female,0
1,1999,0-24,Male,12
2,1999,25-64,Female,119
3,1999,25-64,Male,144
4,1999,65+,Female,1103
...,...,...,...,...
127,2020,0-24,Male,0
128,2020,25-64,Female,243
129,2020,25-64,Male,314
130,2020,65+,Female,1354


Now I save the final cleaned mortality data in the processed data folder 

In [14]:
aggregated_data.to_csv("../Processed Data/1999_2020_Mortality_Data.csv", index=False)

### Dataset 2: ADHS Asthma-Related Inpatient Discharges and Emergency Visits Data

Here I take the manually cleaned file and split it into multiple files to make it more usable.

In [15]:
hospital_data = pd.read_excel('../Processed Data/Hospital_Inpatient_Emergency_Data.xlsx')
hospital_data

,Year,Gender,Type of Record,"Asthma, All Mentions","Asthma, First Diagnosis"
0,2003,Female,Inpatient Discharge,6540,1168
1,2003,Female,Emergency Visits,8591,3358
2,2003,Male,Inpatient Discharge,3517,1066
3,2003,Male,Emergency Visits,6804,3336
4,2004,Female,Inpatient Discharge,13729,2318
...,...,...,...,...,...
71,2020,Male,Emergency Visits,23202,4722
72,2021,Female,Inpatient Discharge,19892,617
73,2021,Female,Emergency Visits,35640,6055
74,2021,Male,Inpatient Discharge,10907,503


We notice that there is a scope for us to split the above dataset into two based on the type of records to make it more usable. Therefore, in the next steps, I do this split and save it as two separate files.

In [16]:
inpatient_discharge_data = hospital_data[hospital_data['Type of Record'] == 'Inpatient Discharge']
inpatient_discharge_data.head()

,Year,Gender,Type of Record,"Asthma, All Mentions","Asthma, First Diagnosis"
0,2003,Female,Inpatient Discharge,6540,1168
2,2003,Male,Inpatient Discharge,3517,1066
4,2004,Female,Inpatient Discharge,13729,2318
6,2004,Male,Inpatient Discharge,7321,1836
8,2005,Female,Inpatient Discharge,14963,2444


In [17]:
emergency_visits_data = hospital_data[hospital_data['Type of Record'] == 'Emergency Visits']
emergency_visits_data.head()

,Year,Gender,Type of Record,"Asthma, All Mentions","Asthma, First Diagnosis"
1,2003,Female,Emergency Visits,8591,3358
3,2003,Male,Emergency Visits,6804,3336
5,2004,Female,Emergency Visits,18366,6273
7,2004,Male,Emergency Visits,14039,6245
9,2005,Female,Emergency Visits,23984,7346


In [18]:
inpatient_discharge_data.to_csv('../Processed Data/2003_2021_inpatient_discharge_data.csv', index=False)
emergency_visits_data.to_csv('../Processed Data/2003_2021_emergency_visits_data.csv', index=False)

### Dataset 3: ADHS Medical Procedures Data

Here, I load the raw medical procedures data and preprocess it to ensure it is in a usable and standardised format. We then store it into two separate files.

This data was also in a very unusable format, and therefore it first required me to understand what all is there in the data and how is it stored, before being able to put it in a standardised format

In [19]:
df = pd.read_excel('../Raw Data/By Procedures/proc300.xls', header=1)
print(df)

   Number of Procedures by Category and County of Residence, Arizona, 2000  \
0                                                  \n                        
1                                                                            
2                                                 NaN                        
3                    Operations on the nervous system                        
4                                          Spinal tap                        
5                               Operations on the eye                        
6                               Operations on the ear                        
7            Operations on the nose mouth and pharynx                        
8                Operations on the respiratory system                        
9                 Bronchoscopy with or without biopsy                        
10            Operations on the cardiovascular system                        
11             Removal of coronary artery obstruction           

From above we can see tht directly reading the file wasn't very useful, therefore after manually looking at the data for a specific year, I decided to attempt reading it by skipping certain rows and doing basic preprocessing.

In [21]:
# Read the Excel file and skip the first 3 rows to get the actual data
df = pd.read_excel('../Raw Data/By Procedures/proc300.xls', skiprows=3, header=1)

df.rename(columns={'Unnamed: 0': 'Procedure', 'Unnamed: 1': 'Total'}, inplace=True)
print(df.head())

                                  Procedure  Total  Apache  Cochise  Coconino  \
0          Operations on the nervous system  22758     136      370       354   
1                                Spinal tap   4739      24       54        65   
2                     Operations on the eye   2995      15       99        47   
3                     Operations on the ear    377       3        7        14   
4  Operations on the nose mouth and pharynx   2603      23       54        62   

   Gila  Graham  Greenlee  La Paz  Maricopa  Mohave  Navajo  Pima  Pinal  \
0   353      74        27      55     14625     557     468  3262    862   
1    44      11         9       9      3165     117      54   678    201   
2   123      13         6      13      1305      51      28  1040     73   
3    27       0         1       1       196       9       9    62     14   
4    45      13         3      13      1552      88      47   456     89   

   Santa Cruz  Yavapai  Yuma  Unknown  
0         122   

Now, we can see that the above table is in a more readable format with the first column as the procedure name, the next column as the total number of treatments, the other columns as specific numbers for each county in Arizona.

Next, I explore what all procedures exist in the data and what procedures are relevant for our purposes. I specifically choose the procedures that could be affected by wildfires or smoke. 

In [ ]:
df['Procedure'] = df['Procedure'].str.strip()
print(df['Procedure'].unique())

['Operations on the nervous system' 'Spinal tap' 'Operations on the eye'
 'Operations on the ear' 'Operations on the nose mouth and pharynx'
 'Operations on the respiratory system'
 'Bronchoscopy with or without biopsy'
 'Operations on the cardiovascular system'
 'Removal of coronary artery obstruction' 'Coronary artery bypass graft'
 'Cardiac catheterization'
 'Insertion, replacement, removal, and revision of pacemaker leads or device'
 'Hemodialysis' 'Operations on the hemic and lymphatic system'
 'Operations on the digestive system'
 'Endoscopy of small intestine with or without biopsy'
 'Endoscopy of large intestine with or without biopsy'
 'Partial excision of large intestine'
 'Appendectomy, excluding incidental' 'Cholecystectomy'
 'Lysis of peritoneal adhesions' 'Operations on the urinary system'
 'Cystoscopy with or without biopsy'
 'Operations on the male genital organs' 'Prostatectomy'
 'Operations on the female genital organs'
 'Oophorectomy and salpingo-oophorectomy'
 'Bila

After looking at the procedures above, I decided that Respiratory therapy and Operations on the respiratory system are the most relevant with regards to our analysis, and therefore in the next step I filter the data for my city Mesa (which falls in the Maricopa county) and by the procedures defined above.

In [24]:
# Filter the dataframe for the specified procedures and column
procedures = ['Respiratory therapy', 'Operations on the respiratory system']
filtered_df = df[df['Procedure'].isin(procedures)][['Procedure', 'Maricopa']]

filtered_df

,Procedure,Maricopa
5,Operations on the respiratory system,7964
45,Respiratory therapy,6788


Now, that I have figured out what data I want and how to filter the data, I decided to write a function that will go through all the data (the ADHS Procedures data has separate files for each year) and provide me with one final dataset containing the year, which was extracted from the name of the file, and then the procedure, and number of procedures for the Maricopa county.

Note: In this process, I realised that not all the data was in the same format, and it required some manual cleaning to ensure the function was able to process the data correctly

In [ ]:
folder_path = '../Raw Data/By Procedures/'
file_pattern = folder_path + '*.xls*'

# Procedures of interest
procedures = ['Respiratory therapy', 'Operations on the respiratory system']

procedure_dfs = {procedure: [] for procedure in procedures}
for file_path in glob.glob(file_pattern):
    # Extract the year from the file name
    file_name = file_path.split('/')[-1]
    year = file_name.split('.')[0][-2:] 
    year = '20' + year  # Convert to full year
    
    # Preprocessing to read the data correctly
    if year in ['2000', '2001', '2002']: # The data format for these years was different from the rest, requiring different steps
        df = pd.read_excel(file_path, skiprows=3, header=1)
        #print(f"Year {year} and df -> {df.head()}")
        df.rename(columns={'Unnamed: 0': 'Procedure', 'Unnamed: 1': 'Total State'}, inplace=True)
    else:
        df = pd.read_excel(file_path, skiprows=2, header=1)
        df.rename(columns={'Procedures by category': 'Procedure'}, inplace=True)
       
    df['Procedure'] = df['Procedure'].str.strip()
    
    # Filter the dataframe for the specified procedures and Maricopa county
    for procedure in procedures:
        filtered_df = df[df['Procedure'] == procedure][['Procedure', 'Maricopa']].copy()
        if not filtered_df.empty:
            filtered_df.rename(columns={'Maricopa': 'Total'}, inplace=True)
            filtered_df['Year'] = year
            procedure_dfs[procedure].append(filtered_df)

# Combine all dataframes for each procedure into a single dataframe and reorder columns for clarity
final_dfs = {procedure: pd.concat(dfs, ignore_index=True) for procedure, dfs in procedure_dfs.items()}
for procedure in final_dfs:
    final_dfs[procedure] = final_dfs[procedure][['Year', 'Total']]

final_dfs

{'Respiratory therapy':     Year  Total
 0   2000   6788
 1   2001   7220
 2   2002   7801
 3   2003  10682
 4   2004   9819
 5   2005  11796
 6   2006  12290
 7   2007  12970
 8   2008  14531
 9   2009  16002
 10  2010  16570
 11  2011  16616
 12  2012  16679
 13  2013  16612
 14  2014  16676
 15  2015  16711,
 'Operations on the respiratory system':     Year  Total
 0   2000   7964
 1   2001   8953
 2   2002   9507
 3   2003  10061
 4   2004  10951
 5   2005  11804
 6   2006  12709
 7   2007  13075
 8   2008  13985
 9   2009  14571
 10  2010  14161
 11  2011  14129
 12  2012  13849
 13  2013  14030
 14  2014  14002
 15  2015  15096
 16  2016  18683
 17  2017  18764
 18  2018  19743
 19  2019  19954
 20  2020  20724}

Now I store the data into two separate files for further analysis

In [31]:
final_df_operations_respiratory_system = final_dfs['Operations on the respiratory system']
final_df_operations_respiratory_system

,Year,Total
0,2000,7964
1,2001,8953
2,2002,9507
3,2003,10061
4,2004,10951
5,2005,11804
6,2006,12709
7,2007,13075
8,2008,13985
9,2009,14571


In [32]:
final_df_respiratory_therapy = final_dfs['Respiratory therapy']
final_df_respiratory_therapy


,Year,Total
0,2000,6788
1,2001,7220
2,2002,7801
3,2003,10682
4,2004,9819
5,2005,11796
6,2006,12290
7,2007,12970
8,2008,14531
9,2009,16002


In [33]:
final_df_respiratory_therapy.to_csv('../Processed Data/2000_2015_respiratory_therapy.csv', index=False)
final_df_operations_respiratory_system.to_csv('../Processed Data/2000_2020_respiratory_system_operations.csv', index=False)

The final data for each of the procedures is now saved in the processed data folder for further analysis